In [1]:
# Las siguientes celdas (from google.colab import files.....) te las dejamos comentadas por si quieres 
# ejecutarlas en google.colab) para subir los archivos oportunos.


In [2]:
# from google.colab import files
# uploaded = files.upload()

In [3]:
# from google.colab import files
# uploaded = files.upload()

In [4]:
# from google.colab import files
# uploaded = files.upload()

In [5]:
# from google.colab import files
# uploaded = files.upload()

In [6]:
# from google.colab import files
# uploaded = files.upload()

In [7]:
# from google.colab import files
# uploaded = files.upload()

In [8]:
# Importes nativo de Python
import re
import random

# Librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Bibliotecas
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder


In [9]:
# Dmitry tenemos todo el código explicado para tontos, no lo tengas en cuenta.

In [10]:
# LECTURA DE CSV´s

In [11]:
df = pd.read_csv("netflix_titles.csv", encoding='latin1')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8809 entries, 0 to 8808
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       8809 non-null   object 
 1   type          8809 non-null   object 
 2   title         8809 non-null   object 
 3   director      6175 non-null   object 
 4   cast          7984 non-null   object 
 5   country       7978 non-null   object 
 6   date_added    8799 non-null   object 
 7   release_year  8809 non-null   int64  
 8   rating        8805 non-null   object 
 9   duration      8806 non-null   object 
 10  listed_in     8809 non-null   object 
 11  description   8809 non-null   object 
 12  Unnamed: 12   0 non-null      float64
 13  Unnamed: 13   0 non-null      float64
 14  Unnamed: 14   0 non-null      float64
 15  Unnamed: 15   0 non-null      float64
 16  Unnamed: 16   0 non-null      float64
 17  Unnamed: 17   0 non-null      float64
 18  Unnamed: 18   0 non-null    

In [12]:
df1 = pd.read_csv("movie_titles_nuevo.csv", encoding='ISO-8859-1')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17769 entries, 0 to 17768
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   1                17769 non-null  int64  
 1   2003             17762 non-null  float64
 2   Dinosaur Planet  17769 non-null  object 
 3   Unnamed: 3       335 non-null    object 
 4   Unnamed: 4       34 non-null     object 
 5   Unnamed: 5       9 non-null      object 
dtypes: float64(1), int64(1), object(4)
memory usage: 833.1+ KB


In [13]:
# LIMPIEZA DF (8809 Peliculas ),DF1 (17769 Peliculas valoradas por usuarios)

In [14]:
df = df.drop(columns=df.columns[12:26]) #columnas que solo tienen nan´h
df = df.drop(columns=['show_id', 'date_added']) #columnas no relevantes para recomendaciones
titulos_columnas = df.columns
#df.to_csv("netflix_titles_drop.csv", index=False) #guardo un nuevo csv quitando las columnas sin ningun valor
#df = pd.read_csv("netflix_titles_drop.csv")
df = df.drop_duplicates()
df[["title"]].applymap(lambda x : x.lower())

,title
0,dick johnson is dead
1,blood & water
2,ganglands
3,jailbirds new orleans
4,kota factory
...,...
8804,zombieland
8805,zoom
8806,zubaan
8807,parasyte: the grey


In [15]:
# Función que modifica la columna "listed_in" (generos de las películas)
def reemplazar_generos(listed_in, generos):
    if isinstance(listed_in, str):
        generos_lista = listed_in.split(', ')
        generos_reemplazados = [generos.get(g, g) for g in generos_lista]
        return ', '.join(generos_reemplazados)
    return listed_in

In [16]:
generos = {
    'thrillers': 'Thrillers',
    'thriller' : 'Thrillers',
    'tv thrillers': 'Thrillers',
    'tv mysteries' : 'Thrillers',
    'teen tv shows' : 'Teen tv shows',
    'children & family movies': 'Family',
    "kids' tv" : 'Family',
    'romantic tv shows' : 'Romance',
    'romantic movies' : 'Romance',
    'romance' : 'Romance',
    'tv action & adventure' : 'Action',
    'action & adventure' : 'Action',
    'action' : 'Action',
    'horror' : 'Horror',
    'horror movies' : 'Horror',
    'tv horror' : 'Horror',
    'cult movies' : 'Classic',
    'classic & cult tv' : 'Classic',
    'classic movies' : 'Classic',
    'drama' : 'Drama',
    'dramas' : 'Drama',
    'tv dramas' : 'Drama',
    'comedies' : 'Comedy',
    'stand-up comedy & talk shows' : 'Comedy',
    'tv comedies' : 'Comedy',
    'stand-up comedy' : 'Comedy',
    'crime tv shows' : 'Crime tv shows',
    'independent movies' : 'Independent movies',
    'tv sci-fi & fantasy' : 'Sci-fi',
    'sci-fi' : 'Sci-fi',
    'sci-fi & fantasy' : 'Sci-fi',
    'music & musicals' : 'Music & musicals',
    'anime features' : 'Anime',
    'anime series' : 'Anime',
    'documentaries' : 'Documentary',
    'docuseries' : 'Documentary',
    'korean tv shows' : 'World movies and shows',
    'british tv shows' : 'World movies and shows',
    'spanish-language tv shows' : 'World movies and shows',
    'international tv shows' : 'World movies and shows',
    'international movies' : 'World movies and shows',
    'reality tv' : 'Reality tv',
    'faith & spirituality' : 'Faith & spirituality',
    'lgbtq movies' : 'Lgbtq movies',
    'science & nature tv' : 'Science & nature tv',
    'sports movies' : 'Sports movies'
        }

In [17]:
df['listed_in'] = df[["listed_in"]].applymap(lambda x : x.lower())
df['listed_in'] = df['listed_in'].apply(reemplazar_generos, args=(generos,))


In [18]:
df1 = df1.drop(columns=df1.columns[3:]) #columnas que solo tienen nan´h
df1 = df1.rename(columns={'1': 'Movie_Id', '2003': 'Year', "Dinosaur Planet": "title"})
df1[["title"]].applymap(lambda x : x.lower())

,title
0,isle of man tt 2004 review
1,character
2,paula abdul's get up & dance
3,the rise and fall of ecw
4,sick
...,...
17764,where the wild things are and other maurice se...
17765,fidel castro: american experience
17766,epoch
17767,the company


In [19]:
# .MERGE DE DF Y DF1 , OBTENCIÓN DE PELÍCULAS COINCIDENTES ENTRE AMBOS DF

In [20]:
df["titleyear"] = df["title"] + " " + df["release_year"].astype(str)

In [21]:
df1["Year"] = df1["Year"].replace([np.inf, -np.inf, np.nan], 0)
df1["Year"] = df1["Year"].astype(int)
df1["titleyear"] = df1["title"] + " " + df1["Year"].astype(str)
df1

,Movie_Id,Year,title,titleyear
0,2,2004,Isle of Man TT 2004 Review,Isle of Man TT 2004 Review 2004
1,3,1997,Character,Character 1997
2,4,1994,Paula Abdul's Get Up & Dance,Paula Abdul's Get Up & Dance 1994
3,5,2004,The Rise and Fall of ECW,The Rise and Fall of ECW 2004
4,6,1997,Sick,Sick 1997
...,...,...,...,...
17764,17766,2002,Where the Wild Things Are and Other Maurice Se...,Where the Wild Things Are and Other Maurice Se...
17765,17767,2004,Fidel Castro: American Experience,Fidel Castro: American Experience 2004
17766,17768,2000,Epoch,Epoch 2000
17767,17769,2003,The Company,The Company 2003


In [22]:
dfm = pd.merge(left = df, right = df1, left_on = "titleyear", right_on = "titleyear", how = "inner")
#dfm.to_csv("merge.csv", index=False)
#dfm = pd.read_csv("merge.csv")

In [23]:
# Lectura de las valoraciones juntamos en un dataframe y filtrado de valoraciones y quedarnos con los valoraciones de las peliculas que nos coinciden

In [24]:
# Las próximas celdas le hacemos una función para la lectura de los .txt  y también dejamos comentamos el código normal por si "peta" tu ordenador.

In [25]:
def lectura_limpieza_txt(data):
    df = pd.read_csv(data, header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
    df['Movie_Id'] = df[df['Rating'].isna()]['Cust_Id'].str.replace(':', '')
    df['Movie_Id'].fillna(method='ffill', inplace=True)
    df.dropna(subset='Rating', inplace=True)
    return df;
lista_txt = ["combined_data_1.txt","combined_data_2.txt","combined_data_3.txt","combined_data_4.txt"]
df2 = lectura_limpieza_txt(lista_txt[0])
df3 = lectura_limpieza_txt(lista_txt[1])
df4 = lectura_limpieza_txt(lista_txt[2])
df5 = lectura_limpieza_txt(lista_txt[3])

In [26]:
# df2 = pd.read_csv('combined_data_1.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
# df2['Movie_Id'] = df2[df2['Rating'].isna()]['Cust_Id'].str.replace(':', '')
# df2['Movie_Id'].fillna(method='ffill', inplace=True)
# df2.dropna(subset='Rating', inplace=True)

In [27]:
# df3 = pd.read_csv('combined_data_2.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
# df3['Movie_Id'] = df3[df3['Rating'].isna()]['Cust_Id'].str.replace(':', '')
# df3['Movie_Id'].fillna(method='ffill', inplace=True)
# df3.dropna(subset='Rating', inplace=True)

In [28]:
# df4 = pd.read_csv('combined_data_3.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
# df4['Movie_Id'] = df4[df4['Rating'].isna()]['Cust_Id'].str.replace(':', '')
# df4['Movie_Id'].fillna(method='ffill', inplace=True)
# df4.dropna(subset='Rating', inplace=True)

In [29]:
# df5 = pd.read_csv('combined_data_4.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
# df5['Movie_Id'] = df5[df5['Rating'].isna()]['Cust_Id'].str.replace(':', '')
# df5['Movie_Id'].fillna(method='ffill', inplace=True)
# df5.dropna(subset='Rating', inplace=True)

In [30]:
 dfra = pd.concat([df2, df3, df4, df5], axis=0, ignore_index=True) # dfra es el data con los 100.000.000 de valoraciones

In [31]:
dfra["Rating"] = dfra["Rating"].astype("int8")
dfra['Movie_Id'] = dfra['Movie_Id'].astype('int64')
movies=dfm['Movie_Id'].tolist()
dfrating = dfra[dfra['Movie_Id'].apply(lambda x: x in movies)]
dfrating["Cust_Id"] = dfrating["Cust_Id"].astype("int64")

C:\Users\usuario\AppData\Local\Temp\ipykernel_10148\2357871464.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfrating["Cust_Id"] = dfrating["Cust_Id"].astype("int64")


In [32]:
#dfrating.to_csv('rating2.csv', index=False)


In [33]:
# LIMPIEZA DEL DFM (DATA DEL MERGE)

In [34]:
dfm.isna().sum()

type            0
title_x         0
director        8
cast            2
country         6
release_year    0
rating          0
duration        0
listed_in       0
description     0
titleyear       0
Movie_Id        0
Year            0
title_y         0
dtype: int64

In [35]:
dfm = dfm.rename(columns={'rating': 'age'})
dfm.rename(columns={'title_x': 'title'}, inplace=True)
dfm = dfm.drop(columns=['titleyear','title_y','Year'])

In [36]:
edades = {
    'TV-MA': 18,
    'TV-14': 14,
    'PG-13': 13,
    'PG': 8,
    'R': 18,
    'TV-PG': 8,
    'G': 3,
    'TV-Y7': 7,
    'TV-G': 3,
    'NR': 0,
    'TV-Y': 3
}
dfm['age'] = dfm['age'].replace(edades)
# En este proyecto no usamos la columna "age", es para el siguiente proyecto.

In [37]:
# DIRECTORES

In [38]:
# # Reemplazamos NaN por "desconocido"
# dfm['director'].fillna('desconocido', inplace=True)

In [39]:
# # Obtenemos una lista de todos los directores únicos
# directores_unicos = set()
# for director in dfm['director'].dropna().values:
#     directores_unicos.update(director.split(", "))

# directores_unicos = list(directores_unicos)
# label_encoder = LabelEncoder()
# label_encoder.fit(directores_unicos)
# dfm['director_codificado'] = dfm['director'].fillna("").apply(lambda x: label_encoder.transform(x.split(", ")) if x else [])

In [40]:
# PAISES

In [41]:
# # Reemplazamos NaN por "desconocido"
# dfm['country'].fillna('desconocido', inplace=True)

In [42]:
# # Dividimos los valores
# dfm['country'] = dfm['country'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])
# df_expanded = dfm.explode('country')

# paises_unicos = df_expanded['country'].unique()

# # Creamos un objeto LabelEncoder y ajustarlo
# label_encoder_paises = LabelEncoder()
# label_encoder_paises.fit(paises_unicos)

# # Añadimos una columna al DataFrame original 'dfm' con los valores numéricos de los países
# dfm['pais_numero'] = dfm['country'].apply(lambda x: label_encoder_paises.transform(x) if isinstance(x, list) else [])


In [43]:
# COLUMNA TYPE (PELICULA O SERIE)

In [44]:
# tipo = {'Movie': 0, 'TV Show': 1} #reemplazamos tv show y pelicula por 0 y 1
# dfm['type'] = dfm['type'].replace(tipo)


In [45]:
# COLUMNA DURACION

In [46]:
# Función para extraer el valor numérico de la duración
# def extraer_numero(duracion):
#     if isinstance(duracion, str):
#         match = re.search(r'\d+', duracion)
#         if match:
#             return int(match.group())
#     return None


# dfm_result['duration'] = dfm_result['duration'].apply(extraer_numero)

In [47]:
# MODELO DE RECOMENDACION

In [48]:
# Función que saca usuario_id aleatorio
def sacar_id_usuario(df):
    usuario = df["Cust_Id"].value_counts()
    filtrado = usuario[usuario > 4]
    filtrado = filtrado.index.tolist()

    return random.sample(filtrado, k=1)

In [49]:
#  dfrating = pd.read_csv('rating2.csv')
# dfmo = pd.read_csv("merge.csv")
# dfre = pd.read_csv('netflix_titles_drop.csv')
dfmo = dfm
dfre = df
id_usuario= sacar_id_usuario(dfrating)
#Juntamos en un dataframe la valoracion del usuario con los datos de la peliculas
df_filtrado = dfrating[dfrating['Cust_Id'] == id_usuario[0]]
dfme = pd.merge(left = df_filtrado, right = dfmo, left_on = "Movie_Id", right_on = "Movie_Id", how = "inner")
print("El número de peliculas valoradas por el usuario es:", dfme.shape[0])

El número de peliculas valoradas por el usuario es: 12


In [50]:
#generacion de los pesos por genero de cada usuario#
categorias_unicas = set()
for genero in dfme['listed_in'].values:
    generos = genero.split(", ")
    categorias_unicas.update(generos)

categorias_unicas = list(categorias_unicas)

datos = []

for row in dfme[dfme["Rating"] > 0]["listed_in"].values:

    categorias_peliculas = list()

    for cat in categorias_unicas:

        if cat in row.split(", "):
            categorias_peliculas.append(1)
        else:
            categorias_peliculas.append(0)

    datos.append(categorias_peliculas)
#categorias_peliculas.append(int(cat in row.split(", ")))

df_generos_peliculas = pd.DataFrame(data = datos, columns = list(categorias_unicas))
df_filtrado.reset_index(drop=True, inplace=True)

weighted_genre_matrix = pd.concat([df_filtrado, df_generos_peliculas], axis = 1)
weighted_genre_matrix = (weighted_genre_matrix[categorias_unicas].values.T * weighted_genre_matrix['Rating'].values).T
weighted_genre_matrix = pd.DataFrame(weighted_genre_matrix, columns = categorias_unicas)

usuario_pesos = weighted_genre_matrix.sum()

usuario_pesos = usuario_pesos / usuario_pesos.sum()

usuario_pesos

Romance         0.100000
Classic         0.088889
Comedy          0.233333
Lgbtq movies    0.044444
Sci-fi          0.033333
Horror          0.044444
Thrillers       0.155556
Drama           0.133333
Action          0.111111
Family          0.055556
dtype: float64

In [51]:
#creo dataframe columnas por cada genero (en los pesos del usuario) y genero puntuacion por genero
datos = []
for row in dfre['listed_in'].values:
    categorias_peliculas = list()
    for cat in categorias_unicas:
        if cat in row.split(", "):
            categorias_peliculas.append(1)
        else:
            categorias_peliculas.append(0)
    datos.append(categorias_peliculas)

df_generos_peliculas = pd.DataFrame(data=datos, columns=list(categorias_unicas))
dfre2 = pd.concat([dfre, df_generos_peliculas], axis=1)
dfre['Puntuacionge'] = (dfre2[categorias_unicas] * usuario_pesos).sum(axis=1)

In [52]:
#generacion de pesos del usuario por director
dfm2=dfme.dropna(subset = ['director'])
dfm2.reset_index(drop=True, inplace=True)
directores = dfm2['director'].unique().tolist()

datos_directores = []

for row in dfm2[dfm2["Rating"] > 0]["director"].values:

    c_directores = []

    for director in directores :


        if director == row:
            c_directores.append(1)
        else:
            c_directores.append(0)


    datos_directores.append(c_directores)


df_directores = pd.DataFrame(data = datos_directores, columns = list(directores))

weighted_genre_matrix2 = pd.concat([dfm2, df_directores], axis = 1)
weighted_genre_matrix2 = (weighted_genre_matrix2[directores].values.T * weighted_genre_matrix2['Rating'].values).T
weighted_genre_matrix2 = pd.DataFrame(weighted_genre_matrix2, columns = directores )
usuario_d = weighted_genre_matrix2.sum()

usuario_d = usuario_d / usuario_d.sum()

usuario_d

Barbra Streisand     0.08
Barry Levinson       0.08
Barry Sonnenfeld     0.06
Gary Fleder          0.10
Garry Marshall       0.10
Dominic Sena         0.04
Lee Tamahori         0.10
Hugh WIlson          0.10
Peter Segal          0.08
Mathieu Kassovitz    0.08
Jonathan Demme       0.08
Jon Turteltaub       0.10
dtype: float64

In [53]:
# generar dataframe columnas por cada director (en los pesos del usuario) y generar puntuacion por director
datos_directores = []

for row in dfre['director'].values:

    c_directores = []

    for director in directores :


        if director == row:
            c_directores.append(1)
        else:
            c_directores.append(0)


    datos_directores.append(c_directores)


df_directores = pd.DataFrame(data=datos_directores, columns=list(directores))
dfre3 = pd.concat([dfre, df_directores], axis=1)
dfre["Puntuaciond"]= (dfre3[directores] * usuario_d).sum(axis=1)

In [54]:
#generacion de pesos del usuario por actor
dfme['cast'] = dfme['cast'].fillna('')
actores_unicos = set()
for actor in dfme['cast'].values:
    actores = actor.split(", ")
    actores_unicos.update(actores)

actores_unicos = list(actores_unicos)



datos = []

for row in dfme[dfme["Rating"] > 0]["cast"].values:

    actores_peliculas = list()

    for cat in actores_unicos:

        if cat in row.split(", "):
            actores_peliculas.append(1)
        else:
            actores_peliculas.append(0)

    datos.append(actores_peliculas)


df_actores_peliculas = pd.DataFrame(data = datos, columns = list(actores_unicos))
df_filtrado.reset_index(drop=True, inplace=True)

weighted_genre_matrix = pd.concat([df_filtrado, df_actores_peliculas], axis = 1)
weighted_genre_matrix = (weighted_genre_matrix[actores_unicos].values.T * weighted_genre_matrix['Rating'].values).T
weighted_genre_matrix = pd.DataFrame(weighted_genre_matrix, columns = actores_unicos)

usuario_ac = weighted_genre_matrix.sum()

usuario_ac = usuario_ac / usuario_ac.sum()

usuario_ac

Jay O. Sanders       0.018315
Dan Hedaya           0.009158
Robert Downey Jr.    0.007326
Halle Berry          0.010989
Ron Vawter           0.007326
                       ...   
Dorian Harewood      0.007326
Lisa Summerour       0.007326
Justin Bartha        0.009158
Rob Reiner           0.009158
Jane Morris          0.009158
Length: 127, dtype: float64

In [55]:
# generar dataframe columnas por cada actor (en los pesos del usuario) y generar puntuacion por actor
datos = []
dfre['cast'] = dfre['cast'].fillna('')
for row in dfre['cast'].values:
    actores_peliculas = list()
    for cat in actores_unicos:
        if cat in row.split(", "):
            actores_peliculas.append(1)
        else:
            actores_peliculas.append(0)
    datos.append(actores_peliculas)

df_actores_peliculas = pd.DataFrame(data=datos, columns=list(actores_unicos))
dfre4 = pd.concat([dfre, df_actores_peliculas], axis=1)
dfre["Puntuacionac"]=(dfre4[actores_unicos] * usuario_ac).sum(axis=1)

In [56]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    if text is None:
        return ""
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum()]
    tokens = [word for word in tokens if word not in stop_words]
    return " ".join(tokens)

# Preprocesa las descripciones
dfre5 = pd.DataFrame()
dfre5['processed_description'] = dfre['description'].apply(preprocess_text)

# Vectoriza las descripciones preprocesadas
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(dfre5['processed_description'])

# Calcula la matriz de similitud de coseno
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Define la función de recomendación
def get_recommendations(movie_titles, similarity_matrix, df):
    sim_scores_suma = [0] * len(df)
    for movie_title in movie_titles:
        idx = df.index[df['title'] == movie_title].tolist()[0]
        sim_scores = list(enumerate(similarity_matrix[idx]))
        for i, score in sim_scores:
            if i != idx:  # Omitir la película misma
                sim_scores_suma[i] += score

    numero_movies = len(movie_titles)
    sim_scores_media = [score / numero_movies for score in sim_scores_suma]


    for i in range(len(df)):
        df.at[i, 'Puntuacion_des'] = sim_scores_media[i]

    return df

In [57]:
if (dfme['Rating'] == 5).any():
    peliculas_5 = dfme.loc[dfme['Rating'] == 5, 'title'].tolist()
    if len(peliculas_5) == 1:
        movie_title = peliculas_5[0]
        dfre['Puntuacion_des'] = 0
        dfre['Puntuacion_des'] = dfre['Puntuacion_des'].astype(float)
        recommendations = get_recommendations([movie_title], similarity_matrix, dfre)
    else:
        dfre['Puntuacion_des'] = 0
        dfre['Puntuacion_des'] = dfre['Puntuacion_des'].astype(float)
        recommendations = get_recommendations(peliculas_5, similarity_matrix, dfre)
else:
    print("No hay ninguna película con un rating de 5.")

In [58]:

dfre = dfre[~dfre['title'].isin(dfme['title'])]
dfre = dfre.copy()

In [59]:
# se normalizan las puntuaciones y se le da un peso de 65 % a genero 20 % a director, 10 a actores y 5 a descripcion
# en caso de no haya valorado ninguna pelicula con un 5 - son 65 % genero 25 % director y 10 % actores

In [60]:
def calcular_puntuacion(row):
    if row["type"] == "Movie":
        if 'Puntuacion_des' in dfre.columns and dfre['Puntuacion_des'].max() != 0:
            return (
                0.68 * row['Puntuacionge'] / dfre['Puntuacionge'].max() +
                0.16 * row["Puntuaciond"] / dfre["Puntuaciond"].max() +
                0.08 * row["Puntuacionac"] / dfre["Puntuacionac"].max() +
                0.08 * row['Puntuacion_des'] / dfre['Puntuacion_des'].max()
            )
        else:
            return (
                0.7 * row['Puntuacionge'] / dfre['Puntuacionge'].max() +
                0.2 * row["Puntuaciond"] / dfre["Puntuaciond"].max() +
                0.1 * row["Puntuacionac"] / dfre["Puntuacionac"].max()
            )
    else:
        if 'Puntuacion_des' in dfre.columns and dfre['Puntuacion_des'].max() != 0:
            return (
                0.76 * row['Puntuacionge'] / dfre['Puntuacionge'].max() +
                0.14 * row["Puntuacionac"] / dfre["Puntuacionac"].max() +
                0.10 * row['Puntuacion_des'] / dfre['Puntuacion_des'].max()
            )
        else:
            return (
                0.8 * row['Puntuacionge'] / dfre['Puntuacionge'].max() +
                0.2 * row["Puntuacionac"] / dfre["Puntuacionac"].max()
            )

dfre["Puntuacion"] = dfre.apply(calcular_puntuacion, axis=1)

In [62]:
recomendaciones = dfre.sort_values(by='Puntuacion', ascending=False)
edades = {
    'TV-MA': 18,
    'TV-14': 14,
    'PG-13': 13,
    'PG': 8,
    'R': 18,
    'TV-PG': 8,
    'G': 3,
    'TV-Y7': 7,
    'TV-G': 3,
    'NR': 0,
    'TV-Y': 3
}

recomendaciones['rating'] = recomendaciones['rating'].replace(edades) # Modificamos la columna "rating" para que se vean las edades recomendadas por película
recomendaciones.head(20)

,type,title,director,cast,country,release_year,rating,duration,listed_in,description,titleyear,Puntuacionge,Puntuaciond,Puntuacionac,Puntuacion_des,Puntuacion
4629,TV Show,Maniac,NaN,"Emma Stone, Jonah Hill, Justin Theroux, Sally ...",United States,2018.0,18,1 Season,"Comedy, Drama, Thrillers",Two struggling strangers connect during a mind...,Maniac 2018,0.522222,0.0,0.000000,0.026936,0.798577
2826,TV Show,On My Block,NaN,"Sierra Capri, Jason Genao, Brett Gray, Diego T...",United States,2020.0,14,3 Seasons,"Romance, Comedy, Drama",In a rough inner-city Los Angeles neighborhood...,On My Block 2020,0.466667,0.0,0.000000,0.031997,0.724973
1275,TV Show,Ginny & Georgia,NaN,"Brianne Howey, Antonia Gentry, Diesel La Torra...",United States,2021.0,14,1 Season,"Romance, Comedy, Drama","Free-spirited Georgia and her two kids, Ginny ...",Ginny & Georgia 2021,0.466667,0.0,0.000000,0.029668,0.721639
1448,TV Show,Cobra Kai,NaN,"Ralph Macchio, William Zabka, Xolo MaridueÃ±a,...",United States,2021.0,14,3 Seasons,"Action, Comedy, Drama",Decades after the tournament that changed thei...,Cobra Kai 2021,0.477778,0.0,0.000000,0.012406,0.713086
1894,TV Show,Emily in Paris,NaN,"Lily Collins, Philippine Leroy-Beaulieu, Ashle...",United States,2020.0,18,1 Season,"Romance, Comedy, Drama","After landing her dream job in Paris, Chicago ...",Emily in Paris 2020,0.466667,0.0,0.000000,0.021870,0.710470
7984,TV Show,Servant of the People,NaN,"Vladimir Zelenskiy, Natalya Sumskaya, Viktor S...",Ukraine,2015.0,14,1 Season,"World movies and shows, Comedy",After a Ukrainian high school teacher's tirade...,Servant of the People 2015,0.466667,0.0,0.009158,0.013564,0.707908
1722,TV Show,DASH & LILY,NaN,"Midori Francis, Austin Abrams, Dante Brown, Tr...",United States,2020.0,14,1 Season,"Romance, Comedy, Drama",Opposites attract at Christmas as cynical Dash...,DASH & LILY 2020,0.466667,0.0,0.000000,0.018611,0.705802
8726,TV Show,When Calls the Heart,NaN,"Erin Krakow, Daniel Lissing, Lori Loughlin, Ma...",United States,2017.0,8,5 Seasons,Drama,"Leaving behind her upper-crust city roots, a w...",When Calls the Heart 2017,0.477778,0.0,0.000000,0.003601,0.700476
1079,TV Show,The Baker and the Beauty,NaN,"Victor Rasuk, Nathalie Kelley, Dan Bucatinsky,...","Israel, United States",2020.0,8,1 Season,"Romance, Comedy, Drama",A Miami baker and a fashionista try to make ro...,The Baker and the Beauty 2020,0.466667,0.0,0.000000,0.011033,0.694949
5284,TV Show,No Tomorrow,NaN,"Joshua Sasse, Tori Anderson, Jonathan Langdon,...",United States,2016.0,8,1 Season,"Romance, Comedy, Drama",Her straitjacketed life turned topsy-turvy by ...,No Tomorrow 2016,0.466667,0.0,0.000000,0.010548,0.694256
